# topic classification on wikipedia based on outgoing links

In [1]:
# to support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# magics and warnings
%load_ext autoreload
%autoreload 2
import warnings; warnings.simplefilter('ignore')

# machinery
import os, codecs, string, random
from numpy.random import seed as random_seed
from numpy.random import shuffle as random_shuffle
import numpy as np
seed = 42 # reproducibility!
random.seed(seed)
random_seed(seed)
np.random.seed(seed)
import nltk, gensim, sklearn, spacy # the armory!
import matplotlib.pyplot as plt
import pyLDAvis.gensim

In [2]:
import pandas as pd

First, using the <a href=https://docs.python.org/3/library/xml.etree.elementtree.html>xml element tree</a> library we parse the xml dump of the simple english wikipedia.

In [3]:
import xml.etree.ElementTree as ET
tree = ET.parse('simplewiki-20171201-pages-articles-multistream.xml')

In [4]:
root = tree.getroot()
root.tag

'{http://www.mediawiki.org/xml/export-0.10/}mediawiki'

Each children of the root element are actually different pages in wikipedia. As we can see below each page has also 4 children which are `title`, `ns`, `id` and `revision`. We mainly want to do topic classification on articles which belong to name space 0 (`ns=0`) which is the main name space of normal wikipedia articles.

In [5]:
root[3].getchildren()

[<Element '{http://www.mediawiki.org/xml/export-0.10/}title' at 0x11ea047c8>,
 <Element '{http://www.mediawiki.org/xml/export-0.10/}ns' at 0x11ea04818>,
 <Element '{http://www.mediawiki.org/xml/export-0.10/}id' at 0x11ea04868>,
 <Element '{http://www.mediawiki.org/xml/export-0.10/}revision' at 0x11ea048b8>]

In [6]:
page = root.find('{http://www.mediawiki.org/xml/export-0.10/}page')
revisions = page.find('{http://www.mediawiki.org/xml/export-0.10/}revision')
for node in revisions.getiterator():
    if node.tag == '{http://www.mediawiki.org/xml/export-0.10/}text':
        print(node.text[:10])

{{monththi


We can see that the main text of wikipedia articles is in the `text` node which is a child of `revision`. So we put document id, title, namespace and the text of each article in a list of dictionaries to make a pandas data frame out of it.

Also we map each article title to its namespace in `article2ns`. we want to later use this dictionary to see which outgoing links in an article are actually referring to an article in namespace 0.

In [7]:
documents = list()
article2ns = dict()
for child in root:
    title = child.find('{http://www.mediawiki.org/xml/export-0.10/}title')
    docID = child.find('{http://www.mediawiki.org/xml/export-0.10/}id')
    ns = child.find('{http://www.mediawiki.org/xml/export-0.10/}ns')
    revisions = child.find('{http://www.mediawiki.org/xml/export-0.10/}revision')
    if revisions != None:
        text = revisions.find('{http://www.mediawiki.org/xml/export-0.10/}text')
        if text.text != None:
            documents.append({'id': docID.text , 'title': title.text, 'namespace': ns.text, 'text': text.text})
            article2ns[title.text] = ns.text
    

In [8]:
wikiDF = pd.DataFrame(documents)

In [9]:
wikiDF.head()

,id,namespace,text,title
0,1,0,{{monththisyear|4}}\n'''April''' is the 4th [[...,April
1,2,0,{{monththisyear|8}}\n'''August''' (Aug.) is th...,August
2,6,0,[[File:Chemin montant dans les hautes herbes -...,Art
3,8,0,{{more sources|date=February 2012}}\n\n: ''Thi...,A
4,9,0,{{dablink|Air is one of the four [[classical e...,Air


In total we have 239639 articles in this corpus.

In [10]:
wikiDF.shape

(239639, 4)

In [11]:
wikiDF.groupby('namespace').id.count()

namespace
0      182776
10      17888
12        123
14      32289
4        5460
6          36
8         537
828       530
Name: id, dtype: int64

By filtering out the documents which are not in namespace 0 we can see that we have 182776 documents in namespace 0.

In [12]:
documentsDF = wikiDF[wikiDF['namespace']=='0']

In [13]:
documentsDF.shape

(182776, 4)

First of all, we should extract the outgoing links in each article. Links are inside double brackets [[...]]. For extracting them we use the regular expression bellow.

In [14]:
import re
def get_out_links(s):
    pat = r'(?<=\[\[).+?(?=\]\])'
    return re.findall(pat,s)

For the links with two names seperated by `|`, it seems that the first name is the actual wiki page name which is being refrenced and the second name is the text of the hyperlink in the text. for now we keep the first name i.e. the actual wiki page name.
Also for the links which are for the captions of the pictures (the ones starting with File) we skip them for now.

In [15]:
def get_main_linkname(l):
    return list(map(lambda x: x.split('|')[0], l))

In [16]:
def remove_file(l):
    return [s for s in l if not s.startswith('File:')]

Finally, we only keep the links which are pointing to another article in namespace 0. Also we keep the links pointing to an article which is not in our dataset.

In [17]:
def article_only(l):
    out=[]
    for s in l:
        try:
            if article2ns[s]=='0':
                out.append(s)
        except:
            KeyError
            out.append(s)
    return out

In [18]:
documentsDF['out_links'] = documentsDF['text'].map(get_out_links).map(get_main_linkname).map(remove_file).map(article_only)

In [19]:
documentsDF['num_out_links'] = documentsDF['out_links'].map(len)

In [20]:
documentsDF.head(10)

,id,namespace,text,title,out_links,num_out_links
0,1,0,{{monththisyear|4}}\n'''April''' is the 4th [[...,April,"[month, year, March, May, day, July, January, ...",731
1,2,0,{{monththisyear|8}}\n'''August''' (Aug.) is th...,August,"[month, year, Gregorian calendar, July, Septem...",382
2,6,0,[[File:Chemin montant dans les hautes herbes -...,Art,"[artist, creativity, drawing, painting, sculpt...",94
3,8,0,{{more sources|date=February 2012}}\n\n: ''Thi...,A,"[letter, alphabet, Article (grammar), A (disam...",27
4,9,0,{{dablink|Air is one of the four [[classical e...,Air,"[classical element, water, earth, fire, Image:...",43
6,12,0,[[Spain]] is divided in 17 parts called '''aut...,Autonomous communities of Spain,"[Spain, United States, 1978, Catalonia, Catala...",54
7,13,0,[[File:Alan Turing Memorial Closer.jpg|right|t...,Alan Turing,"[Order of the British Empire, FRS, London, Wil...",48
8,14,0,{{Infobox musical artist\n| Name = Ala...,Alanis Morissette,"[Ottawa, Ontario, Canada, Guitar, flute, harmo...",77
9,17,0,{{Infobox software\n| name =...,Adobe Illustrator,"[Adobe Systems, Microsoft Windows, macOS, Vect...",34
10,18,0,[[File:Andouille.jpg|right|thumb|160px]]\n[[Fi...,Andouille,"[pork, sausage, meat, fat, intestines, stomach...",13


In total we have 3649587 outgoing links in all of the articles in our corpus.

In [21]:
sum(list(documentsDF['num_out_links']))

3649587

### LDA

We represent our articles as a list of their outgoing links. Therefore we can represent the whole corpus as a list of lists.

In [22]:
docs = list(documentsDF['out_links'])

We first make a dictionary representation of the links in the documents. Each link in an article is represented by the title of the article it is reffering to. Having this dictionary we can represent each document as a bag of links.

As we can see below, we have 818338 unique links in our whole corpus. This means that there are actually a lot of outgoing links in Simple English Wikipedia which are pointing to an article in the big (actual) Wikipedia.

In [71]:
# Create a dictionary representation of the documents
from gensim.corpora import Dictionary
dictionary = Dictionary(docs)

# Bag-of-words(links) representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]


print('Number of unique tokens(links): %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens(links): 818338
Number of documents: 182776


As training LDA on a big corpus like this may take a conciderable amount of time we use logging library in python in order to see the progress of LDA training algorithm. 

In [27]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Finally we train a <a href=https://radimrehurek.com/gensim/models/ldamodel.html>LDA model from gensim</a> library on our corpus. We set the number of topics to be 10 and do 5 passes over our data. The log of the trainig can be seen at the end of the notebook.

In [78]:
# models
from gensim.models import LdaMulticore,ldamodel
params = {'passes': 5, 'random_state': seed}
base_models = dict()

# model = LdaMulticore(corpus=corpus, num_topics=4, id2word=dictionary, workers=6,
#                 passes=params['passes'], random_state=params['random_state'])

model = ldamodel.LdaModel(corpus=corpus, num_topics=10, id2word=dictionary,
                passes=params['passes'], random_state=params['random_state'])




The top 5 terms in the resulting topics is shown below:

In [67]:
model.show_topics(num_words=5)

[(0,
  '0.005*"United States" + 0.003*"Contemporary R&B" + 0.003*"stroke" + 0.002*"New York (state)" + 0.002*"law"'),
 (1,
  '0.019*"Munch Museum" + 0.001*"Rome" + 0.001*"National Gallery (Norway)" + 0.001*"Downtown MRT Line" + 0.001*"Southeastern (train operating company)"'),
 (2,
  '0.014*"municipality" + 0.014*"Switzerland" + 0.014*"France" + 0.013*"Cantons of Switzerland" + 0.011*"United States"'),
 (3,
  '0.005*"Japan" + 0.005*"Association football" + 0.005*"Brazil" + 0.005*"association football" + 0.004*"Bachelor of Arts"'),
 (4,
  '0.003*"Animal" + 0.003*"UNESCO" + 0.002*"species" + 0.002*"genus" + 0.002*"Bachelor of Science"'),
 (5,
  '0.008*"India" + 0.005*"Tamil language" + 0.004*"Russia" + 0.004*"Australia" + 0.003*"Netherlands"'),
 (6,
  '0.011*"Republican Party (United States)" + 0.011*"Democratic Party (United States)" + 0.010*"Americans" + 0.006*"United States" + 0.006*"United States House of Representatives"'),
 (7,
  '0.008*"Prefectures in France" + 0.008*"Germany" + 0

We use the LDA visualisation from gensim to visualize and interpret the resulting 10 topics.

In [69]:
data =  pyLDAvis.gensim.prepare(model, corpus, dictionary)
pyLDAvis.display(data)

/Applications/anaconda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  topic_term_dists = topic_term_dists.ix[topic_order]


As it can be seen in this visualisation, it is not easy to make a clear interpretation for all of the resulting topics. Especially for some the topics which have low term frequencies, it is really hard to interpret them. But there still some topics which can be interpreted clearly. For Example by looking at the term distributions for topic 1, we can see that it contains the party names in the united states and also the names of the former presidents of the United States. Therefore we can conclude that this topic is about (U.S.) politics. 

Here, I list my interpretations for other topics:

1. United States politics
2. It seems that it is about movie industry, actors and artist. But there are also terms about sweden which were represented only in this topic, so we cannot be sure.
3. different countries.
4. countries, states, municipalities, city, etc.
5. mostly about animals and Environment 
6. mixture of hockey and some tv programms
7. not clear, terms have really low frequency in this topic
8. mostly football (the name of different cities and football clubs of Brazil in this topic strengthen this assumption) 
9. mixture of stuff related to museums and churchrs and subway lines. still not really clear.
10. Mostly about wrestling and rugby. (Here the high ranks of Knesset as the Jews prayer place is strange)


### conclosion
 As it can be seen from above interpreting these topics is not easy and it seems that this topic classification is not perfect and can be improved. One important point is that we have some topics that are mixture of unrelated things. This shows that maybe the selection of 10 topics for the whole Wikipedia is not enough. Also for interpreting a topic, the terms which have small differenc between their frequency within the selected topic and theri overall frequency are more important (in the visualisation, when the differens of the height of the red bar and the blue bar for a term is small.). For example in most of the topics we have the name of the countries, but this terms are not really informative because in most of the Wikipedia articles no matter what the title of the article is, there is usually a sentence that say to which country this title is belonging to and therefore there is a link to the Wikipedia page of that country.

There are also some category pages in Wikipedia (namespace 14) which define a category and give the link to the articles which are within that category. For example <a href=https://simple.wikipedia.org/wiki/Category:Living_people>this</a> page gives the links to the pages which are about famous people who are alive now. One idea is that for all links in an article we try to find the category that each link belongs to and then represent that link by its category rather than its actual title. This may make the interpretation easier. However, as it can be seen in the data frame below not all of these categories are really "categories". Some of them are too specific to be called as a category. For example there is a category called "Category:Bern (canton)" and another category called "Bern" which doesn't make sense.

In [77]:
pd.DataFrame(wikiDF[wikiDF['namespace']=='14'].groupby('title').id.count()).sort_values('id')

,id
title,
"Category:""Part of a series on"" templates",1
Category:Musical entertainers navigational templates,1
Category:Musical entertainers navigational boxes,1
Category:Musical entertainers from Toronto,1
"Category:Musical entertainers from Los Angeles, California",1
Category:Musical entertainers from London,1
Category:Musical entertainers by record label,1
Category:Musical entertainers by nationality,1
Category:Musical entertainers,1


In [66]:
# models
from gensim.models import LdaMulticore,ldamodel
params = {'passes': 5, 'random_state': seed}
base_models = dict()

# model = LdaMulticore(corpus=corpus, num_topics=4, id2word=dictionary, workers=6,
#                 passes=params['passes'], random_state=params['random_state'])

model = ldamodel.LdaModel(corpus=corpus, num_topics=10, id2word=dictionary,
                passes=params['passes'], random_state=params['random_state'])





2018-01-01 02:06:44,709 : INFO : using symmetric alpha at 0.1
2018-01-01 02:06:44,710 : INFO : using symmetric eta at 1.2219889581077744e-06
2018-01-01 02:06:44,863 : INFO : using serial LDA version on this node
2018-01-01 02:07:22,886 : INFO : running online (multi-pass) LDA training, 10 topics, 5 passes over the supplied corpus of 182776 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2018-01-01 02:07:22,900 : INFO : PROGRESS: pass 0, at document #2000/182776
2018-01-01 02:07:25,572 : INFO : merging changes from 2000 documents into a model of 182776 documents
2018-01-01 02:07:26,480 : INFO : topic #5 (0.100): 0.008*"United States" + 0.005*"United Kingdom" + 0.003*"France" + 0.002*"Japan" + 0.002*"London" + 0.002*"Italy" + 0.002*"Germany" + 0.002*"Russia" + 0.002*"Canada" + 0.002*"Australia"
2018-01-01 02:07:26,492 : INFO : topic #3 (0.100): 0.059*"Football League Championship" + 0

2018-01-01 02:07:47,245 : INFO : topic #6 (0.100): 0.007*"footballer" + 0.006*"2014" + 0.005*"United States" + 0.005*"2002" + 0.004*"2015" + 0.004*"1997" + 0.003*"1980" + 0.003*"1983" + 0.003*"2017" + 0.003*"1982"
2018-01-01 02:07:47,259 : INFO : topic #4 (0.100): 0.006*"Greek mythology" + 0.003*"Animal" + 0.003*"species" + 0.003*"English language" + 0.003*"WP:MOSNUM" + 0.002*"Asia" + 0.002*"Africa" + 0.002*"Greek language" + 0.002*"Jesus" + 0.002*"Islam"
2018-01-01 02:07:47,275 : INFO : topic #8 (0.100): 0.008*"footballer" + 0.007*"2014" + 0.007*"2015" + 0.005*"2016" + 0.005*"1999" + 0.005*"2017" + 0.004*"2012" + 0.004*"actress" + 0.004*"2013" + 0.004*"actor"
2018-01-01 02:07:47,289 : INFO : topic #0 (0.100): 0.005*"United States" + 0.003*"England" + 0.003*"music" + 0.002*"computer" + 0.002*"United Kingdom" + 0.002*"piano" + 0.002*"metal" + 0.002*"Zeus" + 0.002*"Americans" + 0.002*"Europe"
2018-01-01 02:07:47,318 : INFO : topic diff=0.313187, rho=0.408248
2018-01-01 02:07:47,340 : INF

2018-01-01 02:08:08,998 : INFO : topic diff=0.252900, rho=0.301511
2018-01-01 02:08:09,022 : INFO : PROGRESS: pass 0, at document #24000/182776
2018-01-01 02:08:10,238 : INFO : merging changes from 2000 documents into a model of 182776 documents
2018-01-01 02:08:12,184 : INFO : topic #1 (0.100): 0.002*"blood" + 0.002*"disease" + 0.002*"F.C. Internazionale Milano" + 0.002*"Rome" + 0.002*"Litre" + 0.001*"bacteria" + 0.001*"Osiris" + 0.001*"philosophy" + 0.001*"medicine" + 0.001*"yellow fever"
2018-01-01 02:08:12,199 : INFO : topic #3 (0.100): 0.017*"Serie A" + 0.005*"Eredivisie" + 0.004*"La Liga" + 0.003*"Brazil" + 0.003*"Association football" + 0.002*"Real Madrid C.F." + 0.002*"Juventus F.C." + 0.002*"Germany national football team" + 0.002*"United States Republican Party" + 0.002*"Serie B"
2018-01-01 02:08:12,212 : INFO : topic #0 (0.100): 0.004*"United States" + 0.004*"music" + 0.002*"computer" + 0.002*"piano" + 0.002*"mathematics" + 0.002*"England" + 0.002*"money" + 0.002*"Microsoft 

2018-01-01 02:08:28,659 : INFO : topic #9 (0.100): 0.011*"United States" + 0.010*"2007" + 0.007*"Americans" + 0.005*"singer" + 0.004*"band" + 0.003*"actor" + 0.003*"2006" + 0.003*"guitar" + 0.003*"Japan" + 0.003*"Beyoncé Knowles"
2018-01-01 02:08:28,674 : INFO : topic #5 (0.100): 0.006*"Italy" + 0.006*"France" + 0.005*"Australia" + 0.005*"United Kingdom" + 0.005*"China" + 0.004*"India" + 0.004*"Japan" + 0.004*"United States" + 0.004*"Russia" + 0.004*"Germany"
2018-01-01 02:08:28,702 : INFO : topic diff=0.244410, rho=0.242536
2018-01-01 02:08:28,723 : INFO : PROGRESS: pass 0, at document #36000/182776
2018-01-01 02:08:29,938 : INFO : merging changes from 2000 documents into a model of 182776 documents
2018-01-01 02:08:32,146 : INFO : topic #7 (0.100): 0.013*"Germany" + 0.007*"Scuderia Ferrari" + 0.006*"Ford Motor Company" + 0.004*"England" + 0.004*"North Rhine-Westphalia" + 0.004*"Bavaria" + 0.004*"district" + 0.003*"Baden-Württemberg" + 0.003*"Michael Schumacher" + 0.002*"London"
2018-

2018-01-01 02:08:48,930 : INFO : topic #2 (0.100): 0.028*"municipality" + 0.015*"Belgium" + 0.010*"Switzerland" + 0.010*"United States" + 0.009*"province" + 0.008*"Cantons of Switzerland" + 0.006*"district" + 0.005*"France" + 0.005*"Delaware" + 0.005*"Nintendo"
2018-01-01 02:08:48,959 : INFO : topic diff=0.141148, rho=0.213201
2018-01-01 02:08:48,984 : INFO : PROGRESS: pass 0, at document #46000/182776
2018-01-01 02:08:49,861 : INFO : merging changes from 2000 documents into a model of 182776 documents
2018-01-01 02:08:51,983 : INFO : topic #0 (0.100): 0.003*"Provinces of Italy" + 0.003*"United States" + 0.002*"Regions of Italy" + 0.002*"Microsoft Windows" + 0.002*"music" + 0.002*"Linux" + 0.002*"Comune" + 0.002*"computer" + 0.002*"television" + 0.002*"Debian"
2018-01-01 02:08:52,001 : INFO : topic #1 (0.100): 0.002*"East Flanders" + 0.001*"Litre" + 0.001*"Gothic architecture" + 0.001*"Christianity" + 0.001*"Hockey Hall of Fame" + 0.001*"philosophy" + 0.001*"church" + 0.001*"DNA" + 0.0

...

2018-01-01 03:34:03,261 : INFO : topic #4 (0.100): 0.006*"Animal" + 0.004*"species" + 0.003*"genus" + 0.003*"UNESCO" + 0.003*"Chordate" + 0.002*"television series" + 0.002*"bird" + 0.002*"Australia" + 0.002*"Africa" + 0.002*"Europe"
2018-01-01 03:34:03,271 : INFO : topic #0 (0.100): 0.004*"United States" + 0.003*"disability" + 0.003*"English language" + 0.002*"television program" + 0.002*"military" + 0.002*"blind" + 0.002*"Contemporary R&B" + 0.001*"Wikipedia" + 0.001*"United Kingdom" + 0.001*"Paris"
2018-01-01 03:34:03,280 : INFO : topic #7 (0.100): 0.010*"England" + 0.007*"Canada" + 0.006*"civil parish" + 0.005*"Chicago Blackhawks" + 0.005*"National Hockey League" + 0.005*"Ontario" + 0.004*"Suffolk" + 0.004*"London" + 0.004*"ice hockey" + 0.004*"Germany"
2018-01-01 03:34:03,312 : INFO : topic diff=0.089041, rho=0.101856
2018-01-01 03:34:06,335 : INFO : -13.802 per-word bound, 14287.4 perplexity estimate based on a held-out corpus of 2000 documents with 26447 words
2018-01-01 03:34:06

2018-01-01 03:34:22,551 : INFO : topic #2 (0.100): 0.019*"France" + 0.018*"United States" + 0.009*"Central European Time" + 0.009*"Central European Summer Time" + 0.008*"Departments of France" + 0.008*"Communes of France" + 0.005*"Country" + 0.005*"city" + 0.004*"List of sovereign states" + 0.003*"Virginia"
2018-01-01 03:34:22,563 : INFO : topic #1 (0.100): 0.003*"Rome" + 0.003*"aircraft" + 0.002*"Julian calendar" + 0.002*"Japanese era name" + 0.002*"Nobel Prize in Physiology or Medicine" + 0.002*"engine" + 0.002*"Christianity" + 0.002*"Italy" + 0.002*"Louis-Frédéric" + 0.002*"2012 Summer Paralympics"
2018-01-01 03:34:22,595 : INFO : topic diff=0.074594, rho=0.101856
2018-01-01 03:34:22,616 : INFO : PROGRESS: pass 4, at document #150000/182776
2018-01-01 03:34:23,377 : INFO : merging changes from 2000 documents into a model of 182776 documents
2018-01-01 03:34:25,623 : INFO : topic #0 (0.100): 0.005*"United States" + 0.003*"English language" + 0.002*"stroke" + 0.002*"natural causes" + 

2018-01-01 03:34:37,280 : INFO : topic #9 (0.100): 0.030*"Sweden" + 0.029*"Americans" + 0.015*"United States" + 0.011*"Counties of Sweden" + 0.010*"Municipalities of Sweden" + 0.010*"Provinces of Sweden" + 0.009*"Statistics Sweden" + 0.008*"movie" + 0.007*"California" + 0.006*"New York City"
2018-01-01 03:34:37,292 : INFO : topic #8 (0.100): 0.008*"WWE" + 0.004*"professional wrestling" + 0.003*"English Premiership (rugby union)" + 0.003*"WWE Championship" + 0.002*"Order of the British Empire" + 0.002*"pay-per-view" + 0.002*"Tag team" + 0.002*"Triple H" + 0.002*"The Undertaker" + 0.002*"John Cena"
2018-01-01 03:34:37,322 : INFO : topic diff=0.077259, rho=0.101856
2018-01-01 03:34:40,248 : INFO : -14.450 per-word bound, 22386.3 perplexity estimate based on a held-out corpus of 2000 documents with 28114 words
2018-01-01 03:34:40,249 : INFO : PROGRESS: pass 4, at document #160000/182776
2018-01-01 03:34:40,976 : INFO : merging changes from 2000 documents into a model of 182776 documents
20

2018-01-01 03:34:54,441 : INFO : topic #3 (0.100): 0.008*"Japan" + 0.006*"association football" + 0.006*"Brazil" + 0.005*"Association football" + 0.003*"Real Madrid C.F." + 0.003*"São Paulo" + 0.002*"Rio de Janeiro" + 0.002*"Spain" + 0.002*"Cristiano Ronaldo" + 0.002*"Manchester City F.C."
2018-01-01 03:34:54,453 : INFO : topic #5 (0.100): 0.006*"India" + 0.006*"Sweden" + 0.005*"Spain" + 0.005*"Australia" + 0.005*"Russia" + 0.004*"Italy" + 0.004*"Germany" + 0.003*"World War II" + 0.003*"Soviet Union" + 0.003*"Poland"
2018-01-01 03:34:54,466 : INFO : topic #6 (0.100): 0.014*"Americans" + 0.008*"Republican Party (United States)" + 0.006*"Democratic Party (United States)" + 0.005*"United States House of Representatives" + 0.005*"United States Army" + 0.005*"United States Senate" + 0.005*"United States" + 0.005*"President of the United States" + 0.004*"Ronald Reagan" + 0.003*"Australian Labor Party"
2018-01-01 03:34:54,491 : INFO : topic diff=0.111647, rho=0.101856
2018-01-01 03:34:54,511 

2018-01-01 03:35:08,649 : INFO : topic #1 (0.100): 0.002*"Rome" + 0.001*"Think of the children" + 0.001*"aircraft" + 0.001*"Italy" + 0.001*"Roman Catholic Church" + 0.001*"bacteria" + 0.001*"Christianity" + 0.001*"philosopher" + 0.001*"jurisdiction" + 0.001*"Christian"
2018-01-01 03:35:08,661 : INFO : topic #2 (0.100): 0.014*"Switzerland" + 0.014*"municipality" + 0.013*"Cantons of Switzerland" + 0.012*"France" + 0.011*"United States" + 0.009*"district" + 0.007*"Departments of France" + 0.006*"Communes of France" + 0.005*"Bern (canton)" + 0.005*"Central European Time"
2018-01-01 03:35:08,671 : INFO : topic #7 (0.100): 0.008*"England" + 0.008*"BBC Two" + 0.007*"Grand Prix (TV programme)" + 0.007*"Prefectures in France" + 0.007*"Canada" + 0.007*"Germany" + 0.005*"London" + 0.004*"BBC One" + 0.004*"National Hockey League" + 0.004*"English people"
2018-01-01 03:35:08,700 : INFO : topic diff=0.086053, rho=0.101856
2018-01-01 03:35:11,633 : INFO : -15.749 per-word bound, 55080.8 perplexity es